In [1]:
from core.tasks import process_deep_entries_data
from core.helpers.common import rm_punc_not_nums, rm_stop_words_txt, translate_to_english_txt, compose
from core.classifiers.feature_selector import DocumentFeatureSelector, BigramFeatureSelector
from core.classifiers.NaiveBayes_classifier import NaiveBayesClassifier
import nltk
from nltk.stem.snowball import SnowballStemmer
import random
import langid
   

In [4]:
csv_file_path = '_playground/sample_data/nlp_out.csv'

print('PROCESSING DEEP ENTRIES DATA')
data = process_deep_entries_data(csv_file_path)[:150]
print('DONE')

print('REMOVING PUNC AND STOP WORDS')
stemmer = SnowballStemmer('english')
rm_punc_and_stop = compose(
    rm_punc_not_nums,
    rm_stop_words_txt,
    stemmer.stem # comment this if we don't need stemming
)
data = [(rm_punc_and_stop(str(ex)), l) for (ex, l) in data if langid.classify(str(ex))[0] == 'en']
print('DONE')

print('SHUFFLING DATA')
random.shuffle(data)
print('DONE')

data_len = len(data)
test_len = int(data_len * 0.4)

print('TAKING OUT TEST/TRAIN DATA')
train_data = data[test_len:]
print("length of training data", len(train_data))
test_data = data[:test_len]
print('DONE')

print('COUNTING TAG FREQUENCIES in TRAIN DATA')
d = {}
for ex, l in train_data:
    d[l] = d.get(l, 0) + 1
print(d)
print('DONE')


PROCESSING DEEP ENTRIES DATA
DONE
REMOVING PUNC AND STOP WORDS
DONE
SHUFFLING DATA
DONE
TAKING OUT TEST/TRAIN DATA
length of training data 90
DONE
COUNTING TAG FREQUENCIES in TRAIN DATA
{'WASH': 7, 'Food': 16, 'Nutrition': 11, 'Health': 10, 'NFI': 4, 'Livelihood': 10, 'Protection': 8, 'Shelter': 9, 'Education': 4, 'Agriculture': 2, 'Cross': 4, 'Logistic': 5}
DONE


In [5]:
print('CREATING FEATURE SELECTOR')
# print(freq_words[:200])
# assert False
selector = DocumentFeatureSelector.new(corpus=data)
print('DONE')

# print('CREATING BIGRAM FEATURE SELECTOR')
# selector = BigramFeatureSelector.new(corpus=data, top=2000)
# selector = DocumentFeatureSelector.new(corpus=data, top=2000)
# print('DONE')

print('CREATING CLASSIFIER')
classifier = NaiveBayesClassifier.new(selector, rm_punc_and_stop, train_data)
print('DONE')

print('ACCURACY', classifier.get_accuracy(test_data))

print('CONFUSION MATRIX')
print(classifier.get_confusion_matrix(test_data))

CREATING FEATURE SELECTOR
DONE
CREATING CLASSIFIER
DONE
ACCURACY 0.1694915254237288
CONFUSION MATRIX
           |              L           P       |
           |     E        i        N  r       |
           |     d        v  L     u  o       |
           |     u        e  o     t  t  S    |
           |     c     H  l  g     r  e  h    |
           |  C  a     e  i  i     i  c  e    |
           |  r  t  F  a  h  s     t  t  l  W |
           |  o  i  o  l  o  t  N  i  i  t  A |
           |  s  o  o  t  o  i  F  o  o  e  S |
           |  s  n  d  h  d  c  I  n  n  r  H |
-----------+----------------------------------+
     Cross | <.> .  4  .  .  .  .  .  .  .  . |
 Education |  . <.> 1  .  .  .  .  .  .  .  . |
      Food |  .  .<10> .  .  .  .  .  .  .  . |
    Health |  .  .  5 <.> .  .  .  .  .  .  . |
Livelihood |  .  .  6  . <.> .  .  .  .  .  . |
  Logistic |  .  .  1  .  . <.> .  .  .  .  . |
       NFI |  .  .  3  .  .  . <.> .  .  .  . |
 Nutrition |  .  .  7  .  .  .  . <